# Spóźnienia tramwajów w Krakowie

https://aczepielik.github.io/post/kraktram/

https://github.com/aczepielik/KRKtram/tree/master/reports


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [0]:
'''
df = pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-23.csv')
df.head(10)
df[ df.tripId == 6351558574044883205 ]
df.delay.value_counts(normalize=True)
df.delay.hist(bins=15);
df.delay.describe()

pd.factorize( [
    ' {} {}'.format(7, 'B'),
    ' {} {}'.format(7, 'A'),
    ' {} {}'.format(10, 'C'),
    ' {} {}'.format(10, 'D')
])[0]

df['plannedTime'].dt.hour.value_counts()
df[['plannedTime']].info()
'''

"\ndf = pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-23.csv')\ndf.head(10)\ndf[ df.tripId == 6351558574044883205 ]\ndf.delay.value_counts(normalize=True)\ndf.delay.hist(bins=15);\ndf.delay.describe()\n\npd.factorize( [\n    ' {} {}'.format(7, 'B'),\n    ' {} {}'.format(7, 'A'),\n    ' {} {}'.format(10, 'C'),\n    ' {} {}'.format(10, 'D')\n])[0]\n\ndf['plannedTime'].dt.hour.value_counts()\ndf[['plannedTime']].info()\n"

In [0]:
df = pd.concat([
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-23.csv'),
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-24.csv'),
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-25.csv'),
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-26.csv'),
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-30.csv'),
    pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-31.csv')
])
df.tail(10)

,index,time_stamp,stop,stopName,number,direction,plannedTime,vehicleId,tripId,status,delay,seq_num
72407,2627,2018-08-01 00:08:10,824,Borek Fałęcki I,8,Borek Fałęcki,2018-08-01 00:02:00,6.352185e+18,6351558574044578069,PREDICTED,6,26.0
72408,2635,2018-08-01 00:11:43,112,Stella-Sawickiego,10,Kopiec Wandy,2018-08-01 00:00:00,6.352185e+18,6351558574044627221,PREDICTED,11,18.0
72409,2638,2018-08-01 00:12:58,407,Czyżyny,10,Kopiec Wandy,2018-08-01 00:01:00,6.352185e+18,6351558574044627221,STOPPING,11,19.0
72410,2644,2018-08-01 00:15:20,408,Rondo Czyżyńskie,10,Kopiec Wandy,2018-08-01 00:04:00,6.352185e+18,6351558574044627221,STOPPING,11,20.0
72411,2648,2018-08-01 00:16:55,413,Os.Kolorowe,10,Kopiec Wandy,2018-08-01 00:05:00,6.352185e+18,6351558574044627221,STOPPING,11,21.0
72412,2654,2018-08-01 00:19:18,2744,Plac Centralny im. R.Reagana,10,Kopiec Wandy,2018-08-01 00:08:00,6.352185e+18,6351558574044627221,STOPPING,11,22.0
72413,2657,2018-08-01 00:20:32,424,Os.Na Skarpie,10,Kopiec Wandy,2018-08-01 00:09:00,6.352185e+18,6351558574044627221,PREDICTED,11,23.0
72414,2662,2018-08-01 00:22:31,429,Klasztorna,10,Kopiec Wandy,2018-08-01 00:11:00,6.352185e+18,6351558574044627221,PREDICTED,11,24.0
72415,2665,2018-08-01 00:23:43,2548,Suche Stawy,10,Kopiec Wandy,2018-08-01 00:13:00,6.352185e+18,6351558574044627221,PREDICTED,10,25.0
72416,2673,2018-08-01 00:26:58,449,Bardosa,10,Kopiec Wandy,2018-08-01 00:15:00,6.352185e+18,6351558574044627221,PREDICTED,11,26.0


In [0]:
df.columns

Index(['index', 'time_stamp', 'stop', 'stopName', 'number', 'direction',
       'plannedTime', 'vehicleId', 'tripId', 'status', 'delay', 'seq_num'],
      dtype='object')

In [0]:
df['delay_secs'] = df['delay'].map(lambda x: x*60)
df['direction_cat'] = df['direction'].factorize()[0]
df['vehicleId'].fillna(-1, inplace=True)
df['seq_num'].fillna(-1, inplace=True)

X = df[ ['number', 'stop', 'direction_cat', 'vehicleId', 'seq_num'] ].values
y = df['delay_secs'].values

model = DecisionTreeRegressor(max_depth=20)
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-45.467212882588065, 0.4031579450048447)

In [0]:
df['delay_secs'] = df['delay'].map(lambda x: x*60)
df['direction_cat'] = df['direction'].factorize()[0]
df['vehicleId'].fillna(-1, inplace=True)
df['seq_num'].fillna(-1, inplace=True)
df['plannedTime'] = pd.to_datetime( df['plannedTime'] )
df['hour'] = df['plannedTime'].dt.hour
df['date_hour'] = df['plannedTime'].apply(lambda x: x.replace(minute=0, second=0))
df['hour_volume'] = df.groupby(['date_hour'])['tripId'].transform('nunique')


def gen_id_num_direction(x):
  return '{} {}'.format(x['number'], x['direction'])
#df['number_direction_id'] = df.apply(lambda x: '{} {}'.format(x['number'], x['direction']), axis=1).factorize()[0]
df['number_direction_id'] = df.apply(gen_id_num_direction, axis=1).factorize()[0]

def gen_id_stop_direction(x):
  return '{} {}'.format(x['stop'], x['direction'])
df['number_stop_id'] = df.apply(gen_id_stop_direction, axis=1).factorize()[0]

feats = [
    'number',
    'stop',
    'direction_cat',
    'vehicleId',
    'seq_num',
    'number_direction_id',
    'number_stop_id',
    'hour' #, 'hour_volume' (-45.44779308043779, 0.39165260841811106)
]
X = df[ feats ].values
y = df['delay_secs'].values

#model = RandomForestRegressor(max_depth=20, n_estimators=50, n_jobs=2)
model = DecisionTreeRegressor(max_depth=20, random_state=0)
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-44.10654251514407, 0.34168189705877894)

In [0]:
np.rand(